# Quick Agents Intro

#### Agenda

- Naive Agents Approach
- Single Agent
- Multiple-Agents
- Chaining Agents

In this notebook, we will take a look at how to use Bedrock with Agents to solve some of the tasks that an LLM on it's own might not be capable of doing.


To use agents we need three main components:

* **L**arge **L**anguage **M**odels (**LLMs**)
* **Tools**
* The **Agents** themselves

Let's get started by initializing a LLM:

In [31]:
from utils import print_ww
from utils.helpers import _invoke_agent

In [34]:
from langchain.llms import Bedrock

llm = Bedrock(model_id="anthropic.claude-v2", model_kwargs={"temperature":0, "max_tokens_to_sample":4098, "stop_sequences":["\n\nQuestion:", "\n\nHuman:"]})

In [35]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [36]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

Now we use our `tools` (containing a single tool for now) to initialize an agent:

In [37]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
	agent="zero-shot-react-description",
	tools=tools,
	llm=llm,
	verbose=True,
	max_iterations=3
)

In [38]:
question = 'what is (8.25*3.1)^1.1?'

Lets first look at the correct answer using simple python math

In [39]:
(8.25*3.1)**1.1

35.36691727821723

Lets then take a look at how a vanilla LLM would solve this question

In [40]:
print_ww(llm(question+"\n\nHuman:"))

 Okay, let's break this down step-by-step:
1) 8.25 * 3.1 = 25.575
2) (25.575)^1.1
3) To evaluate an exponent, we multiply the base (25.575) by itself the number of times indicated by
the exponent (1.1).
4) So (25.575)^1.1 = 25.575 * 25.575^0.1
5) 25.575^0.1 = 2.5575 (using exponent rules)
6) Therefore, (8.25*3.1)^1.1 = 25.575 * 2.5575 = 65.3328125

So the final answer is 65.3328125


I've implemented the `_invoke_agent` helper method to add buffers if necessary to avoid Bedrock API Throttling. Agents are by design going to divide our prompts to multiple prompts and pass them to our LLM, which will generate increased load on the LLM.

_Let's try asking some questions:_

In [44]:
_invoke_agent(zero_shot_agent, question)



> Entering new AgentExecutor chain...
 I need to calculate (8.25*3.1)^1.1 step-by-step.
Action: Calculator
Action Input: 8.25*3.1
Observation: Answer: 25.575
Thought: Now I need to raise 25.575 to the power of 1.1.  
Action: Calculator  
Action Input: 25.575^1.1
Observation: Answer: 35.36691727821723
Thought: I have calculated the final answer.
Final Answer: 35.36691727821723

> Finished chain.


Looks good, let's try something else:

What if we ask a non-math question? Something that requires access to external resources?

In [45]:
zero_shot_agent("What's the population of Norway?")



> Entering new AgentExecutor chain...
 I don't know the exact population of Norway off the top of my head, so I should look it up.
Action: Search the internet
Action Input: "Norway population"
Observation: Search the internet is not a valid tool, try one of [Calculator].
Thought:

OutputParserException: Parsing LLM output produced both a final answer and a parse-able action::  The calculator won't help me find the population of Norway. I need to look this up online.
Action: Apologize
Action Input: "I'm sorry, I don't actually have access to the information needed to definitively answer this question. I'm an AI assistant without internet access."
Thought: I don't have enough information to determine the exact population of Norway. The best I can do is provide an estimate based on my limited knowledge.
Final Answer: I don't have sufficient information to provide the exact population of Norway. However, based on my limited knowledge, I estimate it is likely somewhere between 4 and 7 million people. Please let me know if you are able to provide additional information that could help me determine a more precise answer.

Unfortunately we end up hitting an error. That is because the agent is enforcing the use of a tool (this isn't always the case). The LLM only has access to a mathematics tool outside of it's regular pre-trained knowledge. As we know, LLMs will only have access to data up until their training cut-off which might not include the recent population of Norway.

Lets try to use another tool, wikipedia.

In [46]:
!pip install wikipedia

In [48]:
tools = load_tools(
    ['wikipedia', 'llm-math'],
    llm=llm
)

Now we reinitialize the agent with our two tools (`"Calculator"` and `"Wikipedia"`)

In [49]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5
)

In [51]:
_invoke_agent(zero_shot_agent, "What's the population of Norway?")



> Entering new AgentExecutor chain...
 I need to look up the population of Norway on Wikipedia.
Action: Wikipedia
Action Input: Norway population
Observation: Page: List of urban areas in Norway by population
Summary: This is a list of urban areas in Norway by population, with population numbers as of 1 January 2017. For a list of towns and cities in Norway, see this link.
Statistics Norway, the governmental organisation with the task of measuring the Norwegian population, uses the term tettsted (literally "dense place"; meaning urban settlement or urban area), which is defined as a continuous built-up area with a maximum distance of 50 metres between residences unless a greater distance is caused by public areas, cemeteries or similar reasons. The continuously built-up areas in Norway (urban areas) with the highest population are:

Page: Demographics of Norway
Summary: Demographic features of the population of Norway include population density, ethnicity, education level, health of 

In [52]:
_invoke_agent(zero_shot_agent, question)



> Entering new AgentExecutor chain...
 This is a math question, so I should use the calculator.
Action: Calculator
Action Input: (8.25*3.1)^1.1
Observation: Answer: 35.36691727821723
Thought: I now have the final answer from the calculator.
Final Answer: 35.36691727821723

> Finished chain.


---

In [54]:
chaining_question = "I'm making a business plan that involves calculating revenue of the population that will use my new product. I want you to get the population of Norway and divide it by 1000, the result should be multiplied by 2^1.1 to get the total revenue"
_invoke_agent(zero_shot_agent, chaining_question)



> Entering new AgentExecutor chain...
 To get the population of Norway, I should use Wikipedia. Then I need to use the Calculator to do the math operations.
Action: Wikipedia  
Action Input: population of Norway
Observation: Page: Demographics of Norway
Summary: Demographic features of the population of Norway include population density, ethnicity, education level, health of the populace, economic status, religious affiliations and other aspects.

Page: List of urban areas in Norway by population
Summary: This is a list of urban areas in Norway by population, with population numbers as of 1 January 2017. For a list of towns and cities in Norway, see this link.
Statistics Norway, the governmental organisation with the task of measuring the Norwegian population, uses the term tettsted (literally "dense place"; meaning urban settlement or urban area), which is defined as a continuous built-up area with a maximum distance of 50 metres between residences unless a greater distance is cause

In [57]:

5368.857*(2**1.1)

11508.39691350443

In [63]:
case = """
Background:
Egypt has recently approved a new vaccine for a highly infectious disease and is planning a mass vaccination campaign. The government of Egypt has contacted your consulting firm to optimize the distribution of vaccines across its provinces.

Data:
Population of Egypt: Unknown
Number of provinces: 3
Province A: 50 percent
Province B: 20 percent
Province C: 30 percent

Vaccine doses available: 50 million
Number of doses per person: 2
Constraints:
Each province must receive at least enough doses to vaccinate 60 percent of its population.
The remaining doses can be distributed freely.
Objectives:
Calculate the minimum number of doses each province must receive to meet the 60% vaccination threshold.
Determine how many doses are left for flexible distribution after meeting the minimum requirement for each province.
What is the percentage of the total population that can be vaccinated with the given supply?
If a new shipment of 10 million doses arrives, update your distribution plan.
Additional Information:
Assume that 5 percent of the doses will be wasted due to various reasons (e.g., spoilage, breakage).
You may need to fetch real-time data of the country to perform your calculations.
"""

In [65]:
_invoke_agent(zero_shot_agent, case)



> Entering new AgentExecutor chain...
 I need to determine the population of Egypt and each province to calculate the minimum number of doses required per province. I can use Wikipedia to lookup Egypt's population and estimate the province populations based on the percentages given.

Action: Wikipedia
Action Input: Egypt population
Observation: Page: Ancient Egypt
Summary: Ancient Egypt was a civilization in North Africa situated in the Nile Valley. Ancient Egyptian civilization followed prehistoric Egypt and coalesced around 3100 BC (according to conventional Egyptian chronology) with the political unification of Upper and Lower Egypt under Menes (often identified with Narmer). The history of ancient Egypt occurred as a series of stable kingdoms, separated by periods of relative instability known as Intermediate Periods: the Old Kingdom of the Early Bronze Age, the Middle Kingdom of the Middle Bronze Age and the New Kingdom of the Late Bronze Age.
Egypt reached the pinnacle of its p